In [2]:
import bokeh
from bokeh.plotting import show
import matplotlib.pyplot as plt

import flowkit as fk
from pathlib import Path

bokeh.io.output_notebook()
%matplotlib inline

_ = plt.ioff()

Loading BokehJS ...

In [3]:
fk.__version__

'0.9.3'

In [7]:
fcs_path = Path('../example_data/test_data_diamond_01.fcs')
sample = fk.Sample(fcs_path_or_data=fcs_path)  # load the diamond data
sample.original_filename = 'diamond data'  # give it a name

In [8]:
sample.channels

channel_number        pnn pns  png         pne       pnr
0               1  channel_A      1.0  (0.0, 0.0)  262144.0
1               2  channel_B      1.0  (0.0, 0.0)  262144.0

In [9]:
f = sample.plot_scatter(x_label_or_number=1, y_label_or_number=2, source='raw')
show(f)

In [12]:
chan_a_idx = sample.get_channel_index(channel_label_or_number='channel_A')
events_a = sample.get_channel_events(channel_index=chan_a_idx, source='raw')  # extract all the channel_A from all the events 

In [13]:
events_a.shape

(200000,)

In [14]:
events_a.min(), events_a.max()

(0.0, 99999.0)

In [31]:
g_strat = fk.GatingStrategy()  # initialize an empty gating strategy

In [32]:
dim_a = fk.Dimension(  # dimension that is going to be used to define a rectangular gate
    dimension_id="channel_A",  # name of the channel along which to define the Dimension
    range_max=50000,  # the gate will include events < range_max
    compensation_ref="uncompensated",  # this channel Dimension doesnt need compensation
    transformation_ref=None,  # this dimension doesnt need transformation
)
dim_b = fk.Dimension(  # dimension that is going to be used to define a rectangular gate
    dimension_id="channel_B",  # name of the channel along which to define the Dimension
    range_min=50000,  # the gate will include events < range_max
    compensation_ref="uncompensated",  # this channel Dimension doesnt need compensation
    transformation_ref=None,  # this dimension doesnt need transformation
)

In [33]:
rect_top_left_gate = fk.gates.RectangleGate(  # define a rectangular gate
    gate_name="top-left",
    parent_gate_name=None, 
    dimensions=[dim_a, dim_b]
)

In [34]:
g_strat.add_gate(gate=rect_top_left_gate)  # add the gate to the gating strategy

In [35]:
res = g_strat.gate_sample(sample=sample)  # apply gating strategy to the sample

In [37]:
res.report  # see results of the gating. This gate kept 25% of all the cells

sample gate_path gate_name      gate_type quadrant_parent parent  \
0  diamond data   (root,)  top-left  RectangleGate            None   None   

   count  absolute_percent  relative_percent  level  
0  50001           25.0005           25.0005      1

In [38]:
v1 = fk.Vertex((25_000, 75_000))  # define the vertices of a polygon gate
v2 = fk.Vertex((50_000, 75_000))
v3 = fk.Vertex((50_000, 100_000))
v4 = fk.Vertex((25_000, 100_000))
vertices = [v1, v2, v3, v4]

In [39]:
poly_gate = fk.gates.PolygonGate(  # define a polygon gate after the rectangular gate defined above
    gate_name="poly1",
    parent_gate_name="top-left",
    dimensions=[dim_a, dim_b],
    vertices=vertices,
)


In [40]:
g_strat.add_gate(gate=poly_gate)  # add the gate to the gating strategy

In [41]:
res = g_strat.gate_sample(sample=sample)  # apply the gating strategy to the sample

In [42]:
res.report  # see results of the gating. This gate keeps 12.5% of all cells which is 50% of the cells from the parent gate

sample         gate_path gate_name      gate_type quadrant_parent  \
0  diamond data           (root,)  top-left  RectangleGate            None   
1  diamond data  (root, top-left)     poly1    PolygonGate            None   

     parent  count  absolute_percent  relative_percent  level  
0      None  50001           25.0005           25.0005      1  
1  top-left  25000           12.5000           49.9990      2

In [43]:
# define the parameters needed for an ellipsoidal gate
center = [100_000, 50_000]  # center
cov = [[5_000 ** 2, 0], [0, 5_000 ** 2]]  # shape and orientation
dist = 1  # size

In [44]:
ellipse_gate = fk.gates.EllipsoidGate(  # define an ellipsoid gate
    gate_name="ellipse1",
    parent_gate_name=None,  # note that this gate doesnt have a parent
    dimensions=[dim_a, dim_b],
    coordinates=center,
    covariance_matrix=cov,
    distance_square=dist,
)


In [45]:
g_strat.add_gate(gate=ellipse_gate)  # add the gate to the gating strategy

In [46]:
res = g_strat.gate_sample(sample=sample)  # apply the gating strategy to the sample

In [47]:
res.report  # see results of the gating. The ellipsoid gate keeps 3.5% of all cells

sample         gate_path gate_name      gate_type quadrant_parent  \
1  diamond data           (root,)  ellipse1  EllipsoidGate            None   
0  diamond data           (root,)  top-left  RectangleGate            None   
2  diamond data  (root, top-left)     poly1    PolygonGate            None   

     parent  count  absolute_percent  relative_percent  level  
1      None   7070            3.5350            3.5350      1  
0      None  50001           25.0005           25.0005      1  
2  top-left  25000           12.5000           49.9990      2

In [49]:
quad_div1 = fk.QuadrantDivider(
    divider_id="chan-a-div",
    dimension_ref="channel_A",
    compensation_ref="uncompensated",
    transformation_ref=None,
    values=[5000],
)
quad_div2 = fk.QuadrantDivider(
    divider_id="chan-b-div",
    dimension_ref="channel_B",
    compensation_ref="uncompensated",
    transformation_ref=None,
    values=[5000],
)
quad_divs = [quad_div1, quad_div2]

# the 2 dividers above will be used to divide the space into 4 quadrants
quad_1 = fk.gates.Quadrant(
    quadrant_id="chanApos-chanBpos",
    divider_refs=["chan-a-div", "chan-b-div"],
    divider_ranges=[(50_000, None), (50_000, None)],
)
quad_2 = fk.gates.Quadrant(
    quadrant_id="chanApos-chanBneg",
    divider_refs=["chan-a-div", "chan-b-div"],
    divider_ranges=[(50_000, None), (None, 50_000)],
)

quad_3 = fk.gates.Quadrant(
    quadrant_id="chanAneg-chanBpos",
    divider_refs=["chan-a-div", "chan-b-div"],
    divider_ranges=[(None, 50_000), (50_000, None)],
)

quad_4 = fk.gates.Quadrant(
    quadrant_id="chanAneg-chanBneg",
    divider_refs=["chan-a-div", "chan-b-div"],
    divider_ranges=[(None, 50_000), (None, 50_000)],
)
quadrants = [quad_1, quad_2, quad_3, quad_4]  # define the quadrants that will make up the quadrant gate

In [50]:
quad_gate1 = fk.gates.QuadrantGate(  # define the quadrant gate
    gate_name="quadgate1",
    parent_gate_name=None,
    dividers=quad_divs,
    quadrants=quadrants,
)

In [51]:
g_strat.add_gate(gate=quad_gate1)  # add the gate to the gating strategy

In [52]:
res = g_strat.gate_sample(sample=sample)  # apply the gating strategy to the sample

In [53]:
res.report  # see results of the gating. 

sample         gate_path          gate_name      gate_type  \
5  diamond data           (root,)  chanAneg-chanBneg   QuadrantGate   
4  diamond data           (root,)  chanAneg-chanBpos   QuadrantGate   
3  diamond data           (root,)  chanApos-chanBneg   QuadrantGate   
2  diamond data           (root,)  chanApos-chanBpos   QuadrantGate   
1  diamond data           (root,)           ellipse1  EllipsoidGate   
0  diamond data           (root,)           top-left  RectangleGate   
6  diamond data  (root, top-left)              poly1    PolygonGate   

  quadrant_parent    parent  count  absolute_percent  relative_percent  level  
5       quadgate1      None  49999           24.9995           24.9995      1  
4       quadgate1      None  50001           25.0005           25.0005      1  
3       quadgate1      None  50000           25.0000           25.0000      1  
2       quadgate1      None  50000           25.0000           25.0000      1  
1            None      None   7070            3.5350            3.5350      1  
0            None      None  50001           25.0005           25.0005      1  
6            None  top-left  25000           12.5000           49.9990      2